## Testing orvara

In this Jupyter notebook, I'll try to reproduce the results from `orvara` by separating the initialization phase from the computation of the likelihood, without modifying the original code.

This is a necessary step to incorporate `orvara` inside `PyORBIT`

I'm starting form the test case `config_HD159062.ini`
This is how the original .ini files look like:

In [ ]:
import numpy as np

In [ ]:
[data_paths]
# Hipparcos ID of the star, used for fetching absolute astrometry.
# If the Hipparcos ID is invalid the fit will not use HGCA astrometry. 
# In that case you must supply a parallax and parallax_error in [priors].
HipID = 85653
# The file containing the Hipparcos Gaia Catalog of Accelerations.
HGCAFile = HGCA_vEDR3.fits
# The file containing the radial velocity time series for the star.
RVFile = orvara/tests/data/HD159062_RV.dat
# The file containing the relative astrometry for the star.
AstrometryFile = orvara/tests/data/HD159062_relAST.txt
# The path to the Gaia/GOST epochs and scan angles as a csv file.
GaiaDataDir = orvara/tests/data/gaia
# The path to all the Hipparcos (original reduction) intermediate data. Subfolders (as used on the original CD) are automatically handled.
Hip1DataDir = orvara/tests/data/hip1
# The path to all the Hipparcos (Floor van Leeuwen/second reduction) intermediate data. Subfolders (as used on the original DVD) are automatically handled.
Hip2DataDir = orvara/tests/data/hip2
# The file path to the initial parameter guesses (starting conditions for the walkders) to the orbit. Set to None for default guess.
start_file = None

[mcmc_settings]
# Number of temperatures to use in the parallel tempering chain.
ntemps = 10
# Number of walkers. Each walker will have ntemps number of chains.
nwalkers = 100
# Number of planets to fit.
nplanets = 1
# Number of steps contained in each chain.
nstep = 20000
# How much to thin the chain (save every thin-th step).
thin = 50
# Number of threads to use with emcee. Note this built-in parallelization is poor.
nthreads = 1
# True if you want to use the epoch astrometry in GaiaDataDir, Hip1DataDir,
# Hip2DataDir. Hip1 and Hip2 data is combined in a 60/40 mix as described in
# the HGCA paper (Brandt 2018 and 2021). False if not. 
use_epoch_astrometry = True
# Use a separate jitter for each RV instrument?
jit_per_inst = False

[priors_settings]
# priors on primary mass (solar), if prior is not specified, mpri should be inf
mpri = 0.8
mpri_sig = 0.05
minjitter = 1e-5
maxjitter = 1e3

[secondary_gaia]
# If the secondary star is in Gaia, set companion_ID to a nonnegative number
# matching the ID of the companion in the relative astrometry file.  Setting
# companion_ID to a negative number ignores the rest of this.
companion_ID = -1
# The rest of these should be from Gaia in units of mas.
pmra = 0
pmdec = 0
epmra = 100
epmdec = 100
corr_pmra_pmdec = 0

[plotting]
# Path to mcmc chains. This is what's produced by the orbit_fit command.
McmcDataFile = orvara/tests/chains/HD159062B_chain000.fits
# Diagnostic plots to check for convergence.
check_convergence = True
# Define burnin for chains.
burnin = 50
# Which companion to plot?
iplanet = 0

# Name of the target
target = HD159062B
# This is a customized range of epochs you want to plot.
start_epoch = 1990.
end_epoch = 2015.
# Number of random orbits drawn from the posterior distribution you want to plot.
num_orbits = 50
# Define step size for plotting.
num_steps = 1500
# Plot random predicted epoch positions on the Astrometry plot.
predicted_years = 1990,2000,2010,2020,2030
# Prediction of the location of the companion in a future epoch.
position_predict = 2010

# Select which plot you want to generate
Astrometry_orbits_plot = True
Astrometric_prediction_plot = True
RV_orbits_plot = True
RV_plot = True
RV_Instrument = All
Relative_separation_plot = True
Position_angle_plot = True
Proper_motion_plot = True
Proper_motion_separate_plots = False
Corner_plot = True

############# Advanced plotting settings #############
# 1. Set axes limits. Set the upper and lower limit for x and y axes.
set_limit = False
xlim = 1980, 2025
ylim = -2.8,2.8
# Choose the color of the marker for plotting the observed data points.
marker_color = blue

# 2. Turn on/off colorbar. Choose a matplotlib colormap, and choose to color
# code by the secondary mass (msec_solar or msec_jup) or eccentricity (ecc).
use_colorbar = True
colormap = viridis
reference = msec_solar

# 3. Turn on/off the title of the plot? Additionally, if user wants to add
# text somewhere on the plot, enter the text and its x and y position (with
# (0, 0) being the bottom left and (1, 1) being the top right).
show_title = False
add_text = True
text_name = HD 159062B
x_text = 0.5
y_text = 0.9


[save_results]
# quantiles: median and 1sigma = 0.16, 0.5, 0.84, 
# median and 2sigma = 0.025, 0.5, 0.975, or arbitrary
save_params = True
err_margin = 0.16, 0.5, 0.84

First, we have to understand how the config file is parsed inside `orvara`, in order to reproduce it inside `PyORBIT` and take advantage of `orvara` automatic data donwload and initialization

In [ ]:
## this is what is ahappening in orvara
from orvara import orbit
from orvara.config import parse_args
from orvara.format_fits import make_header, pack_cols

from configparser import ConfigParser

standard_config = ConfigParser()
standard_config.read('config_HD159062.ini')

print(standard_config['plotting'])


In [ ]:
import os

"""
files 'data/HGCA_vEDR3.fits', 'data/HD159062_RV.dat',  'data/HD159062_relAST.txt' already there
'data/gaia', 'data/hip1', 'data/hip2' must be created
"""
try:
    os.mkdir('data/gaia')
    os.mkdir('data/hip1')
    os.mkdir('data/hip2')
except:
    pass

config = ConfigParser()

config.add_section('data_paths')
config.set('data_paths', 'HipID', '85653')
config.set('data_paths', 'HGCAFile', 'data/HGCA_vEDR3.fits')
config.set('data_paths', 'RVFile', 'data/HD159062_RV.dat')
config.set('data_paths', 'AstrometryFile', 'data/HD159062_relAST.txt')
config.set('data_paths', 'GaiaDataDir', 'data/gaia')
config.set('data_paths', 'Hip1DataDir', 'data/hip1')
config.set('data_paths', 'Hip2DataDir', 'data/hip2')
config.set('data_paths', 'start_file', 'None')

config.add_section('mcmc_settings')
config.set('mcmc_settings', 'nplanets', '1')
config.set('mcmc_settings', 'use_epoch_astrometry', 'True')


use_epoch_astrometry = True
returninfo = False

In [ ]:
###automatic mode:

'''
import configparser
file =open("employee1.ini","w")
config_object = configparser.ConfigParser()
myDict={'employee': {'name': 'John Doe', 'age': '35'},
        'job': {'title': 'Software Engineer', 'department': 'IT', 'years_of_experience': '10'},
        'address': {'street': '123 Main St.', 'city': 'San Francisco', 'state': 'CA', 'zip': '94102'}}
sections=myDict.keys()
for section in sections:
    config_object.add_section(section)
for section in sections:
    inner_dict=myDict[section]
    fields=inner_dict.keys()
    for field in fields:
        value=inner_dict[field]
        config_object.set(section, field, str(value))
config_object.write(file)
file.close()
'''

From orvara/main.py > run(),

Definitions that we are not going to use have been commented out


In [ ]:
#nwalkers = config.getint('mcmc_settings', 'nwalkers', fallback=100)
#ntemps = config.getint('mcmc_settings', 'ntemps', fallback=10)
#nplanets = config.getint('mcmc_settings', 'nplanets')
#jit_per_inst = config.getboolean('mcmc_settings', 'jit_per_inst', fallback=False)
#nstep = config.getint('mcmc_settings', 'nstep')
#thin = config.getint('mcmc_settings', 'thin', fallback=50)
#nthreads = config.getint('mcmc_settings', 'nthreads', fallback=1)
#use_epoch_astrometry = config.getboolean('mcmc_settings', 'use_epoch_astrometry', fallback=False)
HipID = config.getint('data_paths', 'HipID', fallback=0)
start_file = config.get('data_paths', 'start_file', fallback='none')
#starname = config.get('plotting', 'target', fallback='HIP%d' % (HipID))
#








In [ ]:
'''
[priors_settings]
## priors on primary mass (solar), if prior is not specified, mpri should be inf
mpri = 0.8
mpri_sig = 0.05
minjitter = 1e-5
maxjitter = 1e3
# Not sure if these are relevant for the analysis
'''

config.add_section('priors_settings')
config.set('priors_settings', 'mpri', '0.8')
config.set('priors_settings', 'mpri_sig', '0.05')
config.set('priors_settings', 'minjitter', '1e-5')
config.set('priors_settings', 'maxjitter', '1e3')



from orvara.main import get_priors 
priors = get_priors(config)


In [ ]:

'''
[secondary_gaia]
# If the secondary star is in Gaia, set companion_ID to a nonnegative number
# matching the ID of the companion in the relative astrometry file.  Setting
# companion_ID to a negative number ignores the rest of this.
companion_ID = -1
# The rest of these should be from Gaia in units of mas.
pmra = 0
pmdec = 0
epmra = 100
epmdec = 100
corr_pmra_pmdec = 0
'''

config.add_section('secondary_gaia')
config.set('secondary_gaia', 'companion_ID', '-1')
config.set('secondary_gaia', 'pmra', '0')
config.set('secondary_gaia', 'pmdec', '0')
config.set('secondary_gaia', 'epmra', '100')
config.set('secondary_gaia', 'epmdec', '100')
config.set('secondary_gaia', 'corr_pmra_pmdec', '0')


from orvara.main import get_gaia_catalog_companion 
companion_gaia = get_gaia_catalog_companion(config)


### required by initialize_data
- mcmc_settings: use_epoch_astrometry
- priors_setting: parallax
- priors_setting: parallax_error
  (may not be required)



In [ ]:
from orvara.main import initialize_data

data, H1f, H2f, Gf = initialize_data(config, companion_gaia)


In [ ]:

nplanets = config.getint('mcmc_settings', 'nplanets')
njit = 1


from orvara.main import set_initial_parameters

start_file = config.get('data_paths', 'start_file', fallback='none')
ntemps=10
nwalkers=100

par0 = set_initial_parameters(start_file, ntemps, nplanets, nwalkers, 
                              njit=njit, minjit=priors['minjit'], 
                              maxjit=priors['maxjit'])

print(par0[0,0,:])


In [ ]:
ndim = len(par0[0,0,:])


### ndim = 2 + 7 * nplanets
'''
# Each row: mean and standard deviation of the normal distribution
# used to initialize the parameter given in the comment.  For the
# masses and semimajor axes of the companions, and for the jitter, the
# distributions used are lognormal.  In these cases the standard
# deviations are of the natural log about the distribution peak.  In
# all cases, realizations of the distributions that result in
# unphysical parameters or combinations of parameters will be
# automatically adjusted.
#
   5          0.2   # jitter (m/s, lognormal)
   0.82528 0.08864  # primary mass (Msun)
#
# Companion #1 (indexed as companion 0 in relative astrometry)
#
   0.06308 0.0665   # mass (Msun, lognormal)
   9.9878 0.05929     # semimajor axis (AU, lognormal)
   -0.85571 0.00364   # sqrt(ecc)*sin(omega)
   -0.0448 0.01252   # sqrt(ecc)*cos(omega)
   0.8508 0.05226   # inclination (radians, valid region 0->pi)
   1.5819 0.06806   # long asc node (rad, valid -pi->3pi)
   0.80329 0.12488   # long at reference epoch (rad, valid -pi->3pi)
#
# Companion #2 (indexed as companion 1 in relative astrometry) [if you
# have a second companion, put its guesses below as for companion 1]
# The number of parameters given in this file *must* be a match to the
# number of companions that you specify in the configuration file.  It
# should have the same rows, with the same parameter descriptions, as
# for the first companion listed above.
#
'''

minjit=priors['minjit']
maxjit=priors['maxjit']

bounds = [[0, minjit, maxjit],   # jitter
          [1, 1e-4, 1e3],        # mpri (Solar masses)
          [2, 1e-4, 1e3],        # msec (Solar masses)
          [3, 1e-5, 2e5],        # semimajor axis (AU)
          [4, -1, 1.],      # inclination (radians)
          [5, -1, 1.],      # inclination (radians)
          [6, 1e-5, np.pi],      # inclination (radians)
          [7, -np.pi, 3*np.pi],  # longitude of ascending node (rad)
          [8, -np.pi, 3*np.pi]]  # long at ref epoch (rad)


pyde_bounds = [
          [1e-4, 1],        # mpri (Solar masses)
          [1e-4, 1],        # msec (Solar masses)
          [1e-5, 100],        # semimajor axis (AU
          [-1., 1.,],
          [-1., 1.,],         
          [1e-5, np.pi],      # inclination (radians)
          [-np.pi, 3*np.pi],  # longitude of ascending node (rad)
          [-np.pi, 3*np.pi],
            [minjit, maxjit]]   # jitter]  # long at ref epoch (rad)


print(pyde_bounds)

In [ ]:
from htof.main import Astrometry
from orvara import orbit

## These variables must be defined
#loglkwargs = {'returninfo': False, 'use_epoch_astrometry': use_epoch_astrometry,
#                  'data': data, 'nplanets': nplanets, 'H1f': H1f, 'H2f': H2f, 'Gf': Gf, 'priors': priors, 'njitters': njit}


def lnprob(theta):
    """
    Log likelihood function for the joint parameters
    :param theta:
    :param returninfo:
    :param use_epoch_astrometry:
    :param data:
    :param nplanets:
    :param H1f:
    :param H2f:
    :param Gf:
    :return:
    """  
    njitters= njit
    model = orbit.Model(data)
    lnp = 0

    for i in range(nplanets):
        # Note that params.mpri is really the mass contained in the primary + companions inner to the current planet.
        # params.mpri_true is the real mass of the primary. So params.mpri should really be renamed params.interior_mass
        params = orbit.Params(theta, i, nplanets, data.nInst, njitters)
        lnp = lnp + orbit.lnprior(params, minjit=priors['minjit'],
                                  maxjit=priors['maxjit'])

        if not np.isfinite(lnp):
            model.free()
            params.free()
            return -np.inf

        orbit.calc_EA_RPP(data, params, model)
        orbit.calc_RV(data, params, model)
        if data.n_rel_RV > 0:
            # slightly more performant if we ignore the relative RV calculation call entirely if
            # we do not have any relative RVs to calculate.
            orbit.calc_relRV(data, params, model, i)
        orbit.calc_offsets(data, params, model, i)

        chisq_sec = (params.msec - priors[f'm_secondary{i}'])**2/priors[f'm_secondary{i}_sig']**2
        if chisq_sec > 0:  # If chisq_sec > 0 then a prior was set on the mass of the secondary
            # undo the default 1/m prior that is set in orbit.pyx (see lnprior())
            lnp = lnp - 0.5*chisq_sec + np.log(params.msec)
        # else, don't change lnp at all.

        # Free params if we need to cycle through the next companion
        if i < nplanets - 1:
            params.free()

    if use_epoch_astrometry and data.use_abs_ast:
        orbit.calc_PMs_epoch_astrometry(data, model, H1f, H2f, Gf)
    elif data.use_abs_ast:
        orbit.calc_PMs_no_epoch_astrometry(data, model)

    if returninfo:
        return orbit.calcL(data, params, model, chisq_resids=True, RVoffsets=RVoffsets)

    if np.isfinite(priors['mpri_sig']):
        return lnp - 0.5*(params.mpri_true - priors['mpri'])**2/priors['mpri_sig']**2 + orbit.calcL(data, params, model)
    else:
        return lnp - np.log(params.mpri_true) + orbit.calcL(data, params, model)



def lnprob_modified(theta):
    """
    Log likelihood function for the joint parameters
    :param theta:
    :param returninfo:
    :param use_epoch_astrometry:
    :param data:
    :param nplanets:
    :param H1f:
    :param H2f:
    :param Gf:
    :return:
    """  
    njitters= njit
    model = orbit.Model(data)
    lnp = 0

    for i in range(nplanets):
        # Note that params.mpri is really the mass contained in the primary + companions inner to the current planet.
        # params.mpri_true is the real mass of the primary. So params.mpri should really be renamed params.interior_mass
        params = orbit.Params(theta, i, nplanets, data.nInst, njitters)
        lnp_prior = orbit.lnprior(params, minjit=priors['minjit'],
        #                          maxjit=priors['maxjit'])

        if not np.isfinite(lnp_prior):
            model.free()
            params.free()
            return -np.inf

        orbit.calc_EA_RPP(data, params, model)
        orbit.calc_RV(data, params, model)
        if data.n_rel_RV > 0:
            # slightly more performant if we ignore the relative RV calculation call entirely if
            # we do not have any relative RVs to calculate.
            orbit.calc_relRV(data, params, model, i)
        orbit.calc_offsets(data, params, model, i)

        chisq_sec = (params.msec - priors[f'm_secondary{i}'])**2/priors[f'm_secondary{i}_sig']**2
        if chisq_sec > 0:  # If chisq_sec > 0 then a prior was set on the mass of the secondary
            # undo the default 1/m prior that is set in orbit.pyx (see lnprior())
            lnp = lnp - 0.5*chisq_sec + np.log(params.msec)
        # else, don't change lnp at all.

        # Free params if we need to cycle through the next companion
        if i < nplanets - 1:
            params.free()

    if use_epoch_astrometry and data.use_abs_ast:
        orbit.calc_PMs_epoch_astrometry(data, model, H1f, H2f, Gf)
    elif data.use_abs_ast:
        orbit.calc_PMs_no_epoch_astrometry(data, model)

    return orbit.calcL(data, params, model)


In [ ]:
import multiprocessing
from pyde.de import DiffEvol

num_threads = 6
returninfo = False
with multiprocessing.Pool(num_threads) as pool:

    de = DiffEvol(
        lnprob_modified,
        pyde_bounds,
        nwalkers,
        maximize=True,
        pool=pool)
    de.optimize(100000)

In [ ]:
population = de.population
starting_point = np.nanmedian(population, axis=0)
print(starting_point)


returninfo = True
RVoffsets = True
res, RVoffsets = lnprob(starting_point)
print(
res.plx_best, res.pmra_best, res.pmdec_best,
                                res.chisq_sep, res.chisq_PA,
                                res.chisq_H, res.chisq_HG, res.chisq_G, res.chisq_relRV)

In [ ]:
print(data.use_abs_ast)
print(use_epoch_astrometry)

In [ ]:
10**(0.5*starting_point[-1])


In [ ]:
'''
def lnprior(Params par, double minjit=-20, double maxjit=20):

    cdef extern from "math.h" nogil:
        double sin(double _x)
        double log(double _x)

    cdef int i
    cdef double pi = 3.14159265358979323846264338327950288 # np.pi
    cdef double zeroprior = -np.inf

    if par.sau <= 0 or par.mpri_true <= 0 or par.msec <= 0 or par.ecc >= 1:
        return zeroprior
    if par.sau > 2e5 or par.mpri_true > 1e3 or par.msec > 1e3:
        return zeroprior
    if par.inc < 0 or par.inc > pi or par.asc < -pi or par.asc > 3*pi:
        return zeroprior
    if par.lam < -pi or par.lam > 3*pi:
        return zeroprior
    for i in range(par.ninst_jit):
        if par.all_jit[i] < minjit or par.all_jit[i] > maxjit:
            return zeroprior

    return log(sin(par.inc)*1./(par.sau*par.msec))
'''